In [9]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from statsmodels.tools import add_constant
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import statsmodels.api as sm
import pickle
import os
from sklearn import preprocessing
sns.set(style="ticks")
%matplotlib inline

am = pd.read_csv("boston_pm_LU_and_weather.csv")
def replace_units(df):
    tf = df["Measure Unit"] == ("ug/m3 LC")
    df['Measure Value'] = np.where(tf, df['Measure Value']*0.001, df['Measure Value'])
replace_units(am)
a = am.loc[:, ["Latitude", "Longitude","Sample Collection Start Time", 'Measure Value', 'Cropland',
       'Golf Course', 'Saltwater Sandy Beach', 'cemetary','commercial','Parameter Name','crop_land', 
           'forest', 'high_density_residential', 'industrial',
       'low_density_residential', 'marina', 'medium_density_residential',
       'mining', 'open_land', 'recreational', 'transitional', 'transportation',
           'urban_public/institutional', 'waste', 'water', 'wetland', 'temp_c', 'wind_kph', 'UV']]
'''Rename columns for a consistent naming format'''
def rename_again(df):
    df=df.rename(columns = {'Saltwater Sandy Beach':'saltwater_sandybeach'})
    df=df.rename(columns = {'urban_public/institutional':'urban_public_institution'})
    return df
a=rename_again(a)

In [10]:
"""Delete Variables that do not contribute to ppm"""
#Deletions only need to be made for PM
del a["saltwater_sandybeach"]
del a["recreational"]
del a["marina"]
del a["mining"]
del a["waste"]
del a["cemetary"]
del a["Cropland"]
del a["Golf Course"]
del a["high_density_residential"]
del a["low_density_residential"]
#del am["not_assigned"]

In [11]:
a.columns

Index(['Latitude', 'Longitude', 'Sample Collection Start Time',
       'Measure Value', 'commercial', 'Parameter Name', 'crop_land', 'forest',
       'industrial', 'medium_density_residential', 'open_land', 'transitional',
       'transportation', 'urban_public_institution', 'water', 'wetland',
       'temp_c', 'wind_kph', 'UV'],
      dtype='object')

In [12]:
#renames weather correctly
a.columns = ['Latitude', 'Longitude', 'Sample Collection Start Time',
       'ppm', 'commercial', 'Parameter Name', 'crop_land', 'forest',
       'industrial', 'medium_density_residential', 'open_land', 'transitional',
       'transportation', 'urban_public_institution', 'water', 'wetland',
       'outdoor_temperature', 'wind_speed_resultant', 'solar_radiation']

In [13]:
pm = pd.read_csv("finished_pmweather.csv")
del pm["Unnamed: 0"]
del pm["Sulfation rate"]
del pm["Dew Point"]

In [148]:
def parse_string(str_edit):
    str_edit = str_edit.replace('[', '')
    str_edit = str_edit.replace(' ', '')
#     str_edit = str_edit.replace(' ', ',')
    str_edit = str_edit.replace(']', '')
    list = str_edit.split('-')
    return np.float64(list[0]), (-1.*np.float64(list[1]))
parse_string(pm["Site_keys_x"][0])[0]

30.555472000000002

In [149]:
lat = []
long = []
for i in pm["Site_keys_x"]:
    lat.append(parse_string(i)[0])
    long.append(parse_string(i)[1])
pm["Latitude"] = lat
pm["Longitude"] = long
pm['Sample Collection Start Time'] = 1.0
del pm["Site_keys_x"]

In [150]:
a

,Latitude,Longitude,Sample Collection Start Time,ppm,commercial,Parameter Name,crop_land,forest,industrial,medium_density_residential,open_land,transitional,transportation,urban_public_institution,water,wetland,outdoor_temperature,wind_speed_resultant,solar_radiation
0,42.363100,-71.054200,10.0,0.012,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
1,42.363100,-71.054200,11.0,0.013,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
2,42.363100,-71.054200,12.0,0.016,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
3,42.363100,-71.054200,13.0,0.016,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
4,42.363100,-71.054200,14.0,0.013,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
5,42.363100,-71.054200,15.0,0.010,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
6,42.363100,-71.054200,16.0,0.010,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
7,42.363100,-71.054200,17.0,0.010,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
8,42.363100,-71.054200,18.0,0.012,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524
9,42.363100,-71.054200,19.0,0.010,0.28,Acceptable PM2.5 Aqi and Speciation Mass,0,0.0,0.00,0.0,0.00,0.05,0.19,0.00,0.0,0.0,23.77619,8.497024,1.059524


In [8]:
#reorders columns directly
cols = ['Parameter Name', 'Latitude', 'Longitude', "Sample Collection Start Time", 'ppm', 'forest',
       'open_land', 'water', 'wetland', 'transitional',
       'urban_public_institution', 'commercial', 'transportation', 'crop_land',
       'medium_density_residential', 'industrial',
       'outdoor_temperature', 'solar_radiation', 
       'wind_speed_resultant']

a = a[cols]
a.to_csv("boston_only_pm.csv")

In [154]:
pm = pm[cols]
w = a.append(pm)
w.to_csv("merged_pmweather.csv", index=False)
pd.read_csv("merged_pmweather.csv")

,Parameter Name,Latitude,Longitude,Sample Collection Start Time,ppm,forest,open_land,water,wetland,transitional,urban_public_institution,commercial,transportation,crop_land,medium_density_residential,industrial,outdoor_temperature,solar_radiation,wind_speed_resultant
0,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,10.0,0.012000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
1,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,11.0,0.013000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
2,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,12.0,0.016000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
3,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,13.0,0.016000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
4,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,14.0,0.013000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
5,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,15.0,0.010000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
6,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,16.0,0.010000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
7,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,17.0,0.010000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
8,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,18.0,0.012000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
9,Acceptable PM2.5 Aqi and Speciation Mass,42.363100,-71.054200,19.0,0.010000,0.00000,0.00,0.000000,0.00,0.05,0.000000,0.280000,0.190000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
